# Deep Q-Networks

In this lab we will train a Deep Q-Network (DQN) agent to play Pong task (Atari game) from OpenAI gym.

## 1. Task
The game of Pong is an excellent example of a simple RL task. In the ATARI 2600 version we’ll use you play as one of the paddles (the other is controlled by a decent AI) and you have to bounce the ball past the other player (I don’t really have to explain Pong, right?). On the low level the game works as follows: we receive an image frame (a 210x160x3 byte array (integers from 0 to 255 giving pixel values)) and we get to decide if we want to move the paddle UP or DOWN (i.e. a binary choice). After every single choice the game simulator executes the action and gives us a reward: Either a +1 reward if the ball went past the opponent, a -1 reward if we missed the ball, or 0 otherwise. And of course, our goal is to move the paddle so that we get lots of reward.

<img src=images/pong.gif>


## 2. From Q-Network to Deep Q-Network

We saw that ordinary Q-network was able to barely perform as well as the Q-Table in a simple game environment, Deep Q-Networks are much more capable. In order to transform an ordinary Q-Network into a DQN we will be making the following improvements:
- Going from a single-layer network to a multi-layer convolutional network.
- Implementing Experience Replay, which will allow our network to train itself using stored memories from it’s experience.
- Utilizing a second “target” network, which we will use to compute target Q-values during our updates.

It was these three innovations that allowed the [Google DeepMind team to achieve superhuman performance on dozens of Atari games using their DQN agent](http://www.davidqiu.com:8888/research/nature14236.pdf). We will be walking through each individual improvement.

### Addition 1: Convolutional Layers
Since our agent is going to be learning to play video games, it has to be able to make sense of the game’s screen output in a way that is at least similar to how humans or other intelligent animals are able to. Instead of considering each pixel independently, convolutional layers allow us to consider regions of an image, and maintain spatial relationships between the objects on the screen as we send information up to higher levels of the network. 

### Addition 2: Experience Replay
The second major addition to make DQNs work is Experience Replay. The basic idea is that by storing an agent’s experiences, and then randomly drawing batches of them to train the network, we can more robustly learn to perform well in the task. By keeping the experiences we draw random, we prevent the network from only learning about what it is immediately doing in the environment, and allow it to learn from a more varied array of past experiences. Each of these experiences are stored as a tuple of $\text{<state, action, reward, next_state>}$. The Experience Replay buffer stores a fixed number of recent memories, and as new ones come in, old ones are removed. When the time comes to train, we simply draw a uniform batch of random memories from the buffer, and train our network with them. For our DQN, we have a helper function named `ReplayBuffer` which implements this.

### Addition 3: Separate Target Network
The third major addition to the DQN that makes it unique is the utilization of a second network during the training procedure. This second network is used to generate the target-Q values that will be used to compute the loss for every action during training. Why not use just use one network for both estimations? The issue is that at every step of training, the Q-network’s values shift, and if we are using a constantly shifting set of values to adjust our network values, then the value estimations can easily spiral out of control. The network can become destabilized by falling into feedback loops between the target and estimated Q-values. In order to mitigate that risk, the target network’s weights are fixed, and only periodically or slowly updated to the primary Q-networks values. In this way training can proceed in a more stable manner.

In [1]:
# Imports

import sys
import pickle
import numpy as np
from collections import namedtuple
from itertools import count
import random

import gym
import gym.spaces

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd

from utils.replay_buffer import ReplayBuffer
from utils.gym import get_env, get_wrapper_by_name
from utils.schedule import ConstantSchedule, PiecewiseSchedule, LinearSchedule

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
###########################
# Define Global Variables #
###########################

BATCH_SIZE = 32
GAMMA = 0.99
REPLAY_BUFFER_SIZE = 1000000
LEARNING_STARTS = 50000
LEARNING_FREQ = 4
FRAME_HISTORY_LEN = 10
TARGET_UPDATE_FREQ = 10000
LEARNING_RATE = 0.00025
ALPHA = 0.95
EPS = 0.01
PONG = 3

USE_CUDA = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

class Variable(autograd.Variable):
    def __init__(self, data, *args, **kwargs):
        if USE_CUDA:
            data = data.cuda()
        super(Variable, self).__init__(data, *args, **kwargs)

"""
    OptimizerSpec containing following attributes
        constructor: The optimizer constructor ex: RMSprop
        kwargs: {Dict} arguments for constructing optimizer
"""
OptimizerSpec = namedtuple("OptimizerSpec", ["constructor", "kwargs"])

Statistic = {
    "mean_episode_rewards": [],
    "best_mean_episode_rewards": []
}


In [3]:
##############
# Define DQN #
##############

class DQN(nn.Module):
    def __init__(self, in_channels=4, num_actions=18):
        """
        Initialize a deep Q-learning network as described in
        https://storage.googleapis.com/deepmind-data/assets/papers/DeepMindNature14236Paper.pdf
        Arguments:
            in_channels: number of channel of input.
                i.e The number of most recent frames stacked together as describe in the paper
            num_actions: number of action-value to output, one-to-one correspondence to action in game.
        """
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc4 = nn.Linear(7 * 7 * 64, 512)
        self.fc5 = nn.Linear(512, num_actions)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc4(x.view(x.size(0), -1)))
        return self.fc5(x)


In [6]:
#######################
## TRAINING FUNCTION ##
#######################

def dqn_learing(
    env,
    q_func,
    optimizer_spec,
    exploration,
    stopping_criterion=None,
    replay_buffer_size=1000000,
    batch_size=32,
    gamma=0.99,
    learning_starts=50000,
    learning_freq=4,
    frame_history_len=10,
    target_update_freq=10000
    ):

    """Run Deep Q-learning algorithm.

    You can specify your own convnet using q_func.

    All schedules are w.r.t. total number of steps taken in the environment.

    Parameters
    ----------
    env: gym.Env
        gym environment to train on.
    q_func: function
        Model to use for computing the q function. It should accept the
        following named arguments:
            input_channel: int
                number of channel of input.
            num_actions: int
                number of actions
    optimizer_spec: OptimizerSpec
        Specifying the constructor and kwargs, as well as learning rate schedule
        for the optimizer
    exploration: Schedule (defined in utils.schedule)
        schedule for probability of chosing random action.
    stopping_criterion: (env) -> bool
        should return true when it's ok for the RL algorithm to stop.
        takes in env and the number of steps executed so far.
    replay_buffer_size: int
        How many memories to store in the replay buffer.
    batch_size: int
        How many transitions to sample each time experience is replayed.
    gamma: float
        Discount Factor
    learning_starts: int
        After how many environment steps to start replaying experiences
    learning_freq: int
        How many steps of environment to take between every experience replay
    frame_history_len: int
        How many past frames to include as input to the model.
    target_update_freq: int
        How many experience replay rounds (not steps!) to perform between
        each update to the target Q network
    """
    assert type(env.observation_space) == gym.spaces.Box
    assert type(env.action_space)      == gym.spaces.Discrete

    ###############
    # BUILD MODEL #
    ###############

    if len(env.observation_space.shape) == 1:
        # This means we are running on low-dimensional observations (e.g. RAM)
        input_arg = env.observation_space.shape[0]
    else:
        img_h, img_w, img_c = env.observation_space.shape
        input_arg = frame_history_len * img_c
    num_actions = env.action_space.n

    # Construct an epilson greedy policy with given exploration schedule
    def select_epilson_greedy_action(model, obs, t):
        sample = random.random()
        eps_threshold = exploration.value(t)
        if sample > eps_threshold:
            obs = torch.from_numpy(obs).type(dtype).unsqueeze(0) / 255.0
            # Use volatile = True if variable is only used in inference mode, i.e. don’t save the history
            return model(Variable(obs, volatile=True)).data.max(1)[1].cpu()
        else:
            return torch.IntTensor([[random.randrange(num_actions)]])

    # Initialize target q function and q function
    Q = q_func(input_arg, num_actions).type(dtype)
    target_Q = q_func(input_arg, num_actions).type(dtype)

    # Construct Q network optimizer function
    optimizer = optimizer_spec.constructor(Q.parameters(), **optimizer_spec.kwargs)

    # Construct the replay buffer
    replay_buffer = ReplayBuffer(replay_buffer_size, frame_history_len)

    ###############
    # RUN ENV     #
    ###############
    num_param_updates = 0
    mean_episode_reward = -float('nan')
    best_mean_episode_reward = -float('inf')
    last_obs = env.reset()
    LOG_EVERY_N_STEPS = 10000

    for t in count():
        ### Check stopping criterion
        if stopping_criterion is not None and stopping_criterion(env):
            break

        ### Step the env and store the transition
        # Store lastest observation in replay memory and last_idx can be used to store action, reward, done
        last_idx = replay_buffer.store_frame(last_obs)
        # encode_recent_observation will take the latest observation
        # that you pushed into the buffer and compute the corresponding
        # input that should be given to a Q network by appending some
        # previous frames.
        recent_observations = replay_buffer.encode_recent_observation()

        # Choose random action if not yet start learning
        if t > learning_starts:
            action = select_epilson_greedy_action(Q, recent_observations, t)[0, 0]
        else:
            action = random.randrange(num_actions)
        # Advance one step
        obs, reward, done, _ = env.step(action)
        # clip rewards between -1 and 1
        reward = max(-1.0, min(reward, 1.0))
        # Store other info in replay memory
        replay_buffer.store_effect(last_idx, action, reward, done)
        # Resets the environment when reaching an episode boundary.
        if done:
            obs = env.reset()
        last_obs = obs

        ### Perform experience replay and train the network.
        # Note that this is only done if the replay buffer contains enough samples
        # for us to learn something useful -- until then, the model will not be
        # initialized and random actions should be taken
        if (t > learning_starts and
                t % learning_freq == 0 and
                replay_buffer.can_sample(batch_size)):
            # Use the replay buffer to sample a batch of transitions
            # Note: done_mask[i] is 1 if the next state corresponds to the end of an episode,
            # in which case there is no Q-value at the next state; at the end of an
            # episode, only the current state reward contributes to the target
            obs_batch, act_batch, rew_batch, next_obs_batch, done_mask = replay_buffer.sample(batch_size)
            # Convert numpy nd_array to torch variables for calculation
            obs_batch = Variable(torch.from_numpy(obs_batch).type(dtype) / 255.0)
            act_batch = Variable(torch.from_numpy(act_batch).long())
            rew_batch = Variable(torch.from_numpy(rew_batch))
            next_obs_batch = Variable(torch.from_numpy(next_obs_batch).type(dtype) / 255.0)
            not_done_mask = Variable(torch.from_numpy(1 - done_mask)).type(dtype)

            if USE_CUDA:
                act_batch = act_batch.cuda()
                rew_batch = rew_batch.cuda()

            # Compute current Q value, q_func takes only state and output value for every state-action pair
            # We choose Q based on action taken.
            current_Q_values = Q(obs_batch).gather(1, act_batch.unsqueeze(1))
            # Compute next Q value based on which action gives max Q values
            # Detach variable from the current graph since we don't want gradients for next Q to propagated
            next_max_q = target_Q(next_obs_batch).detach().max(1)[0]
            next_Q_values = not_done_mask * next_max_q
            # Compute the target of the current Q values
            target_Q_values = rew_batch + (gamma * next_Q_values)
            # Compute Bellman error
            bellman_error = target_Q_values - current_Q_values
            # clip the bellman error between [-1 , 1]
            clipped_bellman_error = bellman_error.clamp(-1, 1)
            # Note: clipped_bellman_delta * -1 will be right gradient
            d_error = clipped_bellman_error * -1.0
            # Clear previous gradients before backward pass
            optimizer.zero_grad()
            # run backward pass
            current_Q_values.backward(d_error.data.unsqueeze(1))

            # Perfom the update
            optimizer.step()
            num_param_updates += 1

            # Periodically update the target network by Q network to target Q network
            if num_param_updates % target_update_freq == 0:
                target_Q.load_state_dict(Q.state_dict())

        ### 4. Log progress and keep track of statistics
        episode_rewards = get_wrapper_by_name(env, "Monitor").get_episode_rewards()
        if len(episode_rewards) > 0:
            mean_episode_reward = np.mean(episode_rewards[-100:])
        if len(episode_rewards) > 100:
            best_mean_episode_reward = max(best_mean_episode_reward, mean_episode_reward)

        Statistic["mean_episode_rewards"].append(mean_episode_reward)
        Statistic["best_mean_episode_rewards"].append(best_mean_episode_reward)

        if t % LOG_EVERY_N_STEPS == 0 and t > learning_starts:
            print("Timestep %d" % (t,))
            print("mean reward (100 episodes) %f" % mean_episode_reward)
            print("best mean reward %f" % best_mean_episode_reward)
            print("episodes %d" % len(episode_rewards))
            print("exploration %f" % exploration.value(t))
            sys.stdout.flush()

            # Dump statistics to pickle
            # with open('statistics.pkl', 'wb') as f:
            #     pickle.dump(Statistic, f)
            #     print("Saved to %s" % 'results/statistics.pkl')

In [7]:
##########
## MAIN ##
##########

if __name__ == '__main__':
    
    ## Set Gym
    # Get Atari games.
    atari = gym.benchmark_spec('Atari40M')

    # Change the index to select a different game.
    task = atari.tasks[PONG]
    num_timesteps = task.max_timesteps
    
    ## Run training
    seed = 0        # Use a seed of zero
    env = get_env(task, seed)
    
    def stopping_criterion(env):
        # notice that here t is the number of steps of the wrapped env,
        # which is different from the number of steps in the underlying env
        return get_wrapper_by_name(env, "Monitor").get_total_steps() >= num_timesteps
    
    optimizer_spec = OptimizerSpec(
        constructor=optim.RMSprop,
        kwargs=dict(lr=LEARNING_RATE, alpha=ALPHA, eps=EPS),
    )
    
    exploration_schedule = LinearSchedule(1000000, 0.1)
    
    dqn_learing(
        env=env,
        q_func=DQN,
        optimizer_spec=optimizer_spec,
        exploration=exploration_schedule,
        stopping_criterion=stopping_criterion,
        replay_buffer_size=REPLAY_BUFFER_SIZE,
        batch_size=BATCH_SIZE,
        gamma=GAMMA,
        learning_starts=LEARNING_STARTS,
        learning_freq=LEARNING_FREQ,
        frame_history_len=FRAME_HISTORY_LEN,
        target_update_freq=TARGET_UPDATE_FREQ,
    )



[2017-07-14 17:33:58,332] Making new env: PongNoFrameskip-v4
[2017-07-14 17:33:58,721] Clearing 12 monitor files from previous run (because force=True was provided)
[2017-07-14 17:34:01,539] Starting new video recorder writing to /tmp/summerschool/user10/lab5/results/pong/openaigym.video.0.4125.video000000.mp4
[2017-07-14 17:34:04,087] Starting new video recorder writing to /tmp/summerschool/user10/lab5/results/pong/openaigym.video.0.4125.video000001.mp4
[2017-07-14 17:34:14,503] Starting new video recorder writing to /tmp/summerschool/user10/lab5/results/pong/openaigym.video.0.4125.video000008.mp4
[2017-07-14 17:34:42,761] Starting new video recorder writing to /tmp/summerschool/user10/lab5/results/pong/openaigym.video.0.4125.video000027.mp4
[2017-07-14 17:35:59,732] Starting new video recorder writing to /tmp/summerschool/user10/lab5/results/pong/openaigym.video.0.4125.video000064.mp4


Timestep 60000
mean reward (100 episodes) -20.307692
best mean reward -inf
episodes 65
exploration 0.946000
Timestep 70000
mean reward (100 episodes) -20.328947
best mean reward -inf
episodes 76
exploration 0.937000
Timestep 80000
mean reward (100 episodes) -20.337209
best mean reward -inf
episodes 86
exploration 0.928000
Timestep 90000
mean reward (100 episodes) -20.288660
best mean reward -inf
episodes 97
exploration 0.919000
Timestep 100000
mean reward (100 episodes) -20.340000
best mean reward -20.300000
episodes 108
exploration 0.910000
Timestep 110000
mean reward (100 episodes) -20.380000
best mean reward -20.300000
episodes 119
exploration 0.901000


[2017-07-14 17:40:15,433] Starting new video recorder writing to /tmp/summerschool/user10/lab5/results/pong/openaigym.video.0.4125.video000125.mp4


Timestep 120000
mean reward (100 episodes) -20.360000
best mean reward -20.300000
episodes 130
exploration 0.892000
Timestep 130000
mean reward (100 episodes) -20.300000
best mean reward -20.290000
episodes 140
exploration 0.883000
Timestep 140000
mean reward (100 episodes) -20.290000
best mean reward -20.280000
episodes 151
exploration 0.874000
Timestep 150000
mean reward (100 episodes) -20.310000
best mean reward -20.280000
episodes 162
exploration 0.865000
Timestep 160000
mean reward (100 episodes) -20.300000
best mean reward -20.280000
episodes 173
exploration 0.856000
Timestep 170000
mean reward (100 episodes) -20.260000
best mean reward -20.250000
episodes 184
exploration 0.847000
Timestep 180000
mean reward (100 episodes) -20.270000
best mean reward -20.220000
episodes 195
exploration 0.838000
Timestep 190000
mean reward (100 episodes) -20.240000
best mean reward -20.220000
episodes 205
exploration 0.829000


[2017-07-14 17:46:53,832] Starting new video recorder writing to /tmp/summerschool/user10/lab5/results/pong/openaigym.video.0.4125.video000216.mp4


...

Timestep 740000
mean reward (100 episodes) -18.340000
best mean reward -18.330000
episodes 732
exploration 0.334000
Timestep 750000
mean reward (100 episodes) -18.150000
best mean reward -18.150000
episodes 738
exploration 0.325000
Timestep 760000
mean reward (100 episodes) -17.960000
best mean reward -17.960000
episodes 744
exploration 0.316000
Timestep 770000
mean reward (100 episodes) -17.880000
best mean reward -17.880000
episodes 749
exploration 0.307000
Timestep 780000
mean reward (100 episodes) -17.750000
best mean reward -17.750000
episodes 754
exploration 0.298000
Timestep 790000
mean reward (100 episodes) -17.610000
best mean reward -17.610000
episodes 760
exploration 0.289000
Timestep 800000
mean reward (100 episodes) -17.490000
best mean reward -17.490000
episodes 766
exploration 0.280000
Timestep 810000
mean reward (100 episodes) -17.520000
best mean reward -17.470000
episodes 771
exploration 0.271000
Timestep 820000
mean reward (100 episodes) -17.490000
best mean reward -

[2017-07-14 19:37:01,027] Starting new video recorder writing to /tmp/summerschool/user10/lab5/results/pong/openaigym.video.0.4125.video001000.mp4


Timestep 1360000
mean reward (100 episodes) -11.680000
best mean reward -11.680000
episodes 1002
exploration 0.100000
Timestep 1370000
mean reward (100 episodes) -11.550000
best mean reward -11.550000
episodes 1005
exploration 0.100000
Timestep 1380000
mean reward (100 episodes) -11.630000
best mean reward -11.550000
episodes 1009
exploration 0.100000
Timestep 1390000
mean reward (100 episodes) -11.710000
best mean reward -11.550000
episodes 1014
exploration 0.100000
Timestep 1400000
mean reward (100 episodes) -11.630000
best mean reward -11.550000
episodes 1018
exploration 0.100000
Timestep 1410000
mean reward (100 episodes) -11.380000
best mean reward -11.380000
episodes 1021
exploration 0.100000
Timestep 1420000
mean reward (100 episodes) -11.330000
best mean reward -11.280000
episodes 1024
exploration 0.100000
Timestep 1430000
mean reward (100 episodes) -11.110000
best mean reward -11.110000
episodes 1028
exploration 0.100000
Timestep 1440000
mean reward (100 episodes) -11.000000
b

Timestep 2070000
mean reward (100 episodes) -1.840000
best mean reward -1.840000
episodes 1217
exploration 0.100000
Timestep 2080000
mean reward (100 episodes) -1.570000
best mean reward -1.570000
episodes 1219
exploration 0.100000
Timestep 2090000
mean reward (100 episodes) -1.230000
best mean reward -1.230000
episodes 1222
exploration 0.100000
Timestep 2100000
mean reward (100 episodes) -1.040000
best mean reward -1.040000
episodes 1224
exploration 0.100000
Timestep 2110000
mean reward (100 episodes) -0.640000
best mean reward -0.640000
episodes 1227
exploration 0.100000
Timestep 2120000
mean reward (100 episodes) -0.510000
best mean reward -0.510000
episodes 1230
exploration 0.100000
Timestep 2130000
mean reward (100 episodes) -0.490000
best mean reward -0.440000
episodes 1233
exploration 0.100000
Timestep 2140000
mean reward (100 episodes) -0.370000
best mean reward -0.370000
episodes 1235
exploration 0.100000
Timestep 2150000
mean reward (100 episodes) -0.300000
best mean reward -

KeyboardInterrupt: 

### Visualization of learning

In [ ]:
plt.plot(Statistic["mean_episode_rewards"])

In [ ]:
plt.plot(Statistic["best_mean_episode_rewards"])

## Excercise

0. Observe the performance of the DQN at various training phases in the results folder.
1. Change the hyperparameters, and observe the performance of DQN.
2. Use different `Schedules` (`Constant`, `Piecewise`) and compare them with `LinearSchedule`. (Codes for these are already included, you just need to replace `LinearSchedule` with them.
3. Apply different exploration strategies like `greedy`, `random` and check perfromance of the DQN.

## Acknowledgement

The helper functions have been adapted/copied from Berkely deep learning course [homework 3](https://github.com/berkeleydeeprlcourse/homework/tree/master/hw3)